# 卷积神经网络（convolutional neural network）

In [2]:
import numpy as np
import torch # 基础库
from torch.utils.data import TensorDataset,DataLoader # 数据加载
from torch import nn # 网络模型
from torch.nn import init # 参数初始化
import torch.optim as optim # 优化算法
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE' # plt绘图报错（不显示图的问题）
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

In [2]:
# 计算准确率
def evaluate_accuracy(test_data_iter, model):
    acc_sum, n = 0.0, 0
    for X, y in test_data_iter:
        acc_sum += (model(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [3]:
# trainTransform = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     normTransform
# ])
# testTransform = transforms.Compose([
#     transforms.ToTensor(),
#     normTransform
# ])


mnist_train = torchvision.datasets.FashionMNIST(root='F:/bigdata/ai/Pytorch_Datasets',
                                                train=True,
                                                download=True,
                                                transform=transforms.Compose([transforms.ToTensor()]))
mnist_test = torchvision.datasets.FashionMNIST(root='F:/bigdata/ai/Pytorch_Datasets',
                                               train=False,
                                               download=True,
                                               transform=transforms.Compose([transforms.ToTensor()]))


model = models.alexnet(num_classes = 10)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [5]:
# AlexNet
# -> 第一个卷积层nn.Conv2d(
# in_channels=3,
# out_channels=64,
# kernel_size=11,
# stride=4,
# padding=2),
# outsize = (w-kernel_size+2*padding)/stride  + 1
# 那么input的size要是(in_channels, w,h)
# 卷积层对于图像是没有尺寸限制要求的, 但是in_channels一定要一致

img = torch.randn((1,1,300,300))
filters = torch.randn(1, 1, 11, 11)
o = nn.functional.conv2d(img,filters,stride=5,padding=0)
print(o.size()) # conv2d是下界

p = nn.functional.max_pool2d(o,kernel_size= 3,stride=2)
print(p.size()) # conv2d是上界

avp = nn.functional.adaptive_avg_pool2d(o,output_size= (8,8))
print(avp.size()) # 指定输出output_size

torch.Size([1, 1, 58, 58])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 8, 8])


In [4]:
# print(mnist_test[0][0].size())
# print(mnist_test[0][0].repeat(1,3, 3, 3).size())
# print(mnist_test[0][0].repeat(1,3, 3, 3).flatten().size())
# def show_fashion_mnist(images, labels):
#     # 这里的_表示我们忽略（不使用）的变量
#     _, figs = plt.subplots(1, len(images), figsize=(12, 12))
#     for f, img, lbl in zip(figs, images, labels):
#         f.imshow(img.view((84, 84)).numpy()) # img.view((84, 84)).numpy()
#         f.set_title(lbl)
#         f.axes.get_xaxis().set_visible(False)
#         f.axes.get_yaxis().set_visible(False)
#     plt.show()
# show_fashion_mnist([mnist_test[0][0].repeat(1,1, 3, 3),mnist_test[1][0].repeat(1,1, 3, 3)], [mnist_test[0][1],mnist_test[1][1]])

In [5]:
batch_size = 10

# Create data loaders.
train_dataloader = DataLoader(mnist_train, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(mnist_test, batch_size=batch_size)

num_epochs = 10
for epoch in range(1, num_epochs + 1):
    model.train()
    for X, y in train_dataloader:
        # X.size()-> [1,1,28,28]
        X = X.repeat(1,3, 1, 1) # X = X.repeat(1, 3, 3, 3)
        print(X.size())
        # 计算预测误差
        output = model(X)
        # y = torch.nn.functional.one_hot(y,10) # 不需要
        l = loss(output, y)

        # 反向传播
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    model.eval()
    print('epoch %d, loss: %f, acc: %f' % (epoch, l.item() , evaluate_accuracy(test_dataloader,model)))



torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 28, 28])
torch.Size([10, 3, 2

KeyboardInterrupt: 

In [ ]:
for i in range(20):
    print(model(mnist_test[i][0]).argmax(dim=1) ,"---实际：", mnist_test[i][1] )
